# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 6 2023 11:10AM,258716,19,9234985,"Innogenix, LLC",Released
1,Mar 6 2023 11:10AM,258716,19,9234986,"Innogenix, LLC",Released
2,Mar 6 2023 11:10AM,258716,19,9234987,"Innogenix, LLC",Released
3,Mar 6 2023 11:10AM,258716,19,9234988,"Innogenix, LLC",Released
4,Mar 6 2023 11:10AM,258716,19,9234989,"Innogenix, LLC",Released
5,Mar 6 2023 11:10AM,258716,19,9234990,"Innogenix, LLC",Released
6,Mar 6 2023 11:10AM,258716,19,9235003,"Innogenix, LLC",Released
7,Mar 6 2023 11:10AM,258716,19,9235004,"Innogenix, LLC",Released
8,Mar 6 2023 11:10AM,258716,19,9235005,"Innogenix, LLC",Released
9,Mar 6 2023 11:10AM,258716,19,9235006,"Innogenix, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
13,258707,Executing,1
14,258709,Released,3
15,258711,Released,2
16,258715,Released,1
17,258716,Released,23


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
258707,1.0,NaN
258709,NaN,3.0
258711,NaN,2.0
258715,NaN,1.0
258716,NaN,23.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
258681,5.0,7.0
258688,0.0,1.0
258692,16.0,8.0
258694,1.0,0.0
258695,3.0,26.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
258681,5,7
258688,0,1
258692,16,8
258694,1,0
258695,3,26


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,258681,5,7
1,258688,0,1
2,258692,16,8
3,258694,1,0
4,258695,3,26


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,258681,5,7
1,258688,,1
2,258692,16,8
3,258694,1,
4,258695,3,26


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 6 2023 11:10AM,258716,19,"Innogenix, LLC"
23,Mar 6 2023 11:04AM,258715,10,ISDIN Corporation
24,Mar 6 2023 10:53AM,258711,10,"Citieffe, Inc."
26,Mar 6 2023 10:53AM,258709,10,"Virtue Rx, LLC"
29,Mar 6 2023 10:32AM,258705,16,Sartorius Stedim Filters Inc.
30,Mar 6 2023 10:32AM,258704,10,Eywa Pharma Inc.
38,Mar 6 2023 10:31AM,258702,19,"GUSA Granules USA, Inc."
39,Mar 6 2023 10:30AM,258707,22,"NBTY Global, Inc."
40,Mar 6 2023 10:29AM,258703,12,Hush Hush
41,Mar 6 2023 10:16AM,258697,19,VITABIOTICS USA INC.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Mar 6 2023 11:10AM,258716,19,"Innogenix, LLC",,23
1,Mar 6 2023 11:04AM,258715,10,ISDIN Corporation,,1
2,Mar 6 2023 10:53AM,258711,10,"Citieffe, Inc.",,2
3,Mar 6 2023 10:53AM,258709,10,"Virtue Rx, LLC",,3
4,Mar 6 2023 10:32AM,258705,16,Sartorius Stedim Filters Inc.,,1
5,Mar 6 2023 10:32AM,258704,10,Eywa Pharma Inc.,,8
6,Mar 6 2023 10:31AM,258702,19,"GUSA Granules USA, Inc.",,1
7,Mar 6 2023 10:30AM,258707,22,"NBTY Global, Inc.",1,
8,Mar 6 2023 10:29AM,258703,12,Hush Hush,1,
9,Mar 6 2023 10:16AM,258697,19,VITABIOTICS USA INC.,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 6 2023 11:10AM,258716,19,"Innogenix, LLC",23,
1,Mar 6 2023 11:04AM,258715,10,ISDIN Corporation,1,
2,Mar 6 2023 10:53AM,258711,10,"Citieffe, Inc.",2,
3,Mar 6 2023 10:53AM,258709,10,"Virtue Rx, LLC",3,
4,Mar 6 2023 10:32AM,258705,16,Sartorius Stedim Filters Inc.,1,
5,Mar 6 2023 10:32AM,258704,10,Eywa Pharma Inc.,8,
6,Mar 6 2023 10:31AM,258702,19,"GUSA Granules USA, Inc.",1,
7,Mar 6 2023 10:30AM,258707,22,"NBTY Global, Inc.",,1
8,Mar 6 2023 10:29AM,258703,12,Hush Hush,,1
9,Mar 6 2023 10:16AM,258697,19,VITABIOTICS USA INC.,2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 6 2023 11:10AM,258716,19,"Innogenix, LLC",23,
1,Mar 6 2023 11:04AM,258715,10,ISDIN Corporation,1,
2,Mar 6 2023 10:53AM,258711,10,"Citieffe, Inc.",2,
3,Mar 6 2023 10:53AM,258709,10,"Virtue Rx, LLC",3,
4,Mar 6 2023 10:32AM,258705,16,Sartorius Stedim Filters Inc.,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Mar 6 2023 11:10AM,258716,19,"Innogenix, LLC",23.0,NaN
1,Mar 6 2023 11:04AM,258715,10,ISDIN Corporation,1.0,NaN
2,Mar 6 2023 10:53AM,258711,10,"Citieffe, Inc.",2.0,NaN
3,Mar 6 2023 10:53AM,258709,10,"Virtue Rx, LLC",3.0,NaN
4,Mar 6 2023 10:32AM,258705,16,Sartorius Stedim Filters Inc.,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 6 2023 11:10AM,258716,19,"Innogenix, LLC",23.0,0.0
1,Mar 6 2023 11:04AM,258715,10,ISDIN Corporation,1.0,0.0
2,Mar 6 2023 10:53AM,258711,10,"Citieffe, Inc.",2.0,0.0
3,Mar 6 2023 10:53AM,258709,10,"Virtue Rx, LLC",3.0,0.0
4,Mar 6 2023 10:32AM,258705,16,Sartorius Stedim Filters Inc.,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1810903,48.0,8.0
12,258703,0.0,1.0
15,258692,8.0,16.0
16,258705,1.0,0.0
19,776115,26.0,0.0
22,517401,0.0,2.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1810903,48.0,8.0
1,12,258703,0.0,1.0
2,15,258692,8.0,16.0
3,16,258705,1.0,0.0
4,19,776115,26.0,0.0
5,22,517401,0.0,2.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,48.0,8.0
1,12,0.0,1.0
2,15,8.0,16.0
3,16,1.0,0.0
4,19,26.0,0.0
5,22,0.0,2.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,48.0
1,12,Released,0.0
2,15,Released,8.0
3,16,Released,1.0
4,19,Released,26.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,22
Status,,,,,,
Executing,8.0,1.0,16.0,0.0,0.0,2.0
Released,48.0,0.0,8.0,1.0,26.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,22
0,Executing,8.0,1.0,16.0,0.0,0.0,2.0
1,Released,48.0,0.0,8.0,1.0,26.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,22
0,Executing,8.0,1.0,16.0,0.0,0.0,2.0
1,Released,48.0,0.0,8.0,1.0,26.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()